# Introduction to atomman: Stacking Fault Generation

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction

This Notebook outlines the StackingFault class which generates atomic systems that can be used to evaluate stacking fault energies.  See the [04.6 Gamma Surface Plotting Notebook](04.6. Gamma Surface Plotting.ipynb) for analysis and plotting of computed stacking fault energies.

*New as of version 1.2.6*

**Library Imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.2.7
Notebook executed on 2019-06-05


## 2. Underlying theory

### 2.1 Basic system design

Atomic configurations investigating stacking faults are constructed by

1. Starting with a unit cell system.

2. Generating a new supercell system by rotating the unit cell such that the slip plane being investigated is perpendicular to one of the three Cartesian vectors, and only one box vector of the rotated system has a component out of the slip plane.  See the [04.4. Free Surface Rotation Basis Notebook](04.4. Free Surface Rotation Basis.ipynb) for one way to identify an appropriate set of rotation vectors.

3. The boundary conditions are kept periodic along the two box vectors within the slip plane and made non-periodic along the box vector with the out-of-plane component.

4. All atoms above the defined slip plane are displaced by the same shift vector, $\mathbf{d}$.

With the conditions of step 2, there are only three options for how to construct the generic system. The cutboxvector parameter controls this:

- cutboxvector = 'a': the slip plane is perpendicular to the x-axis, and only the $\mathbf{a}$ box vector has a component in the x direction.

- cutboxvector = 'b': the slip plane is perpendicular to the y-axis, and only the $\mathbf{b}$ box vector has a component in the y direction.

- cutboxvector = 'c': the slip plane is perpendicular to the z-axis, and only the $\mathbf{c}$ box vector has a component in the z direction.

The area of the fault plane, $A_{fault}$, can be computed using the two box vectors that are not the cut box vector. For example, if the cut box vector is $\mathbf{c}$, then 

$$A_{fault} = \left| \mathbf{a} \times \mathbf{b} \right|$$

### 2.2 Slip vector

As the slip plane can be explored in 2 dimensions, the applied shift, $\mathbf{d}$, can be expressed using two non-parallel vectors within the fault plane, $\mathbf{a_1}$ and $\mathbf{a_2}$. Taking $\mathbf{a_1}$ and $\mathbf{a_2}$ as lattice slip vectors (i.e. vectors between two symmetrically identical lattice sites) will define a 2D cell encompassing all possible unique stacking fault shifts for the fault plane.  $\mathbf{d}$ can then be represented using fractional coordinates, $a_1$ and $a_2$, relative to $\mathbf{a_1}$ and $\mathbf{a_2}$

$$ \mathbf{d} = a_1 \mathbf{a_1} + a_2 \mathbf{a_2} $$

One complication related to defining the $\mathbf{a_1}$ and $\mathbf{a_2}$ slip vectors is that we typically think of their values in crystallographic Miller or Miller-Bravais indices, but the shifting vector $\mathbf{d}$ needs to be in Cartesian units corresponding to the atomic system.  To better facilitate this difference, the class allows for a reference unit cell box and transformation matrix to also be given.  This allows for the $\mathbf{a_1}$ and $\mathbf{a_2}$ vectors to be given in Miller or Miller-Bravais indices relative to the unit cell box, and then transformed from the unit cell's Cartesian axes to the system's Cartesian axes.

## 3. Class methods

### 3.1 StackingFault initialization

Initializes an object for generating generalized stacking fault atomic configurations.

Parameters
        
- __system__ (*atomman.System*) The atomic system to generate stacking fault configurations from. The system should be oriented such that the stacking fault plane is parallel to either the xy-plane, the xz-plane or the yz-plane.

- __a1vect__ (*array-like object, optional*) A slip vector within the slip plane.  Depending on if ucellbox and transform are given, this can be either a Miller crystal vector or a Cartesian vector relative to the supplied system.  If a1vect is not given and a2vect is, then a1vect is set to [0,0,0].

- __a2vect__ (*array-like object, optional*) A slip vector within the slip plane.  Depending on if ucellbox and transform are given, this can be either a Miller crystal vector or a Cartesian vector relative to the supplied system.  If a2vect is not given and a1vect is, then a2vect is set to [0,0,0].

- __ucellbox__ (*atomman.Box, optional*) If ucellbox is given, then the a1vect and a2vect slip vectors are taken as Miller crystal vectors relative to ucellbox.  If not given, then the slip vectors are taken as Cartesian vectors.

- __transform__ (*array-like object, optional*) A transformation tensor to apply to the a1vect and a2vect slip vectors.  This is needed if system is oriented differently than ucellbox, i.e. system is rotated.

- __cutboxvector__ (*str, optional*) Defines which of the three box vectors gets cut so that the slip plane can be added.  'a' indicates the slip plane is parallel to the yz-plane, 'b' the xz-plane and 'c' the xy-plane.  Note that  the specified cutboxvector is the only box vector allowed to have a component out of the slip plane.

- __faultposrel__ (*float, optional*) The position to place the slip plane within the system given as a relative coordinate along the out-of-plane direction.  faultposrel and faultposcart cannot both be given.  Default value is 0.5 if faultposcart is also not given.

- __faultposcart__ (*float, optional*) The position to place the slip plane within the system given as a Cartesian coordinate along the out-of-plane direction.  faultposrel and faultposcart cannot both be given.

- __vacuumwidth__ (*float, optional*) If given, the out-of-plane dimension of system is increased by this amount to create a vacuum region.  Useful if the generated configurations are to be evaluated with DFT.

### 3.2 StackingFault.set()

Sets generation parameters associated with a stacking fault.

Parameters
        
- __system__ (*atomman.System*) The atomic system to generate stacking fault configurations from. The system should be oriented such that the stacking fault plane is parallel to either the xy-plane, the xz-plane or the yz-plane.

- __a1vect__ (*array-like object, optional*) A slip vector within the slip plane.  Depending on if ucellbox and transform are given, this can be either a Miller crystal vector or a Cartesian vector relative to the supplied system.  If a1vect is not given and a2vect is, then a1vect is set to [0,0,0].

- __a2vect__ (*array-like object, optional*) A slip vector within the slip plane.  Depending on if ucellbox and transform are given, this can be either a Miller crystal vector or a Cartesian vector relative to the supplied system.  If a2vect is not given and a1vect is, then a2vect is set to [0,0,0].

- __ucellbox__ (*atomman.Box, optional*) If ucellbox is given, then the a1vect and a2vect slip vectors are taken as Miller crystal vectors relative to ucellbox.  If not given, then the slip vectors are taken as Cartesian vectors.

- __transform__ (*array-like object, optional*) A transformation tensor to apply to the a1vect and a2vect slip vectors.  This is needed if system is oriented differently than ucellbox, i.e. system is rotated.

- __cutboxvector__ (*str, optional*) Defines which of the three box vectors gets cut so that the slip plane can be added.  'a' indicates the slip plane is parallel to the yz-plane, 'b' the xz-plane and 'c' the xy-plane.  Note that  the specified cutboxvector is the only box vector allowed to have a component out of the slip plane.

- __faultposrel__ (*float, optional*) The position to place the slip plane within the system given as a relative coordinate along the out-of-plane direction.  faultposrel and faultposcart cannot both be given.  Default value is 0.5 if faultposcart is also not given.

- __faultposcart__ (*float, optional*) The position to place the slip plane within the system given as a Cartesian coordinate along the out-of-plane direction.  faultposrel and faultposcart cannot both be given.

- __vacuumwidth__ (*float, optional*) If given, the out-of-plane dimension of system is increased by this amount to create a vacuum region.  Useful if the generated configurations are to be evaluated with DFT.

### 3.3 StackingFault.fault()

Generates a fault configuration by displacing all atoms above the slip plane.

Parameters
        
- __a1__ (*float, optional*) The fractional coordinate of a1vect to shift by.  Default value is 0.0.

- __a2__ (*float, optional*) The fractional coordinate of a2vect to shift by.  Default value is 0.0. 

- __outofplane__ (*float, optional*) An out-of-plane shift, given in absolute units.  Default value is 0.0.

- __faultshift__ (*array-like object, optional*) The full shifting vector to displace the atoms above the slip plane by.  Cannot be given with a1, a2, or outofplane.

- __minimum_r__ (*float, optional*) *Added version 1.2.7* Specifies the minimum allowed interatomic spacing across the slip plane.  If any sets of atoms are closer than this value then the outofplane shift is increased.  Default value is None, which performs no adjustment.

Returns
        
- (*atomman.System*) The atomic configuration with stacking fault shift

### 3.4 StackingFault.iterfaultmap()

Iterates over generalized stacking fault configurations associated with a 2D map of equally spaced a1, a2 coordinates.

Parameters
        
- __num_a1__ (*int*) The number of a1 values to generate systems for.  Default value is 1 (only generate for a1=0.0).

- __num_a2__ (*int*) The number of a2 values to generate systems for.  Default value is 1 (only generate for a2=0.0).

- __outofplane__ (*float, optional*) An out-of-plane shift, given in absolute units.  Default value is 0.0.

- __minimum_r__ (*float, optional*) *Added version 1.2.7* Specifies the minimum allowed interatomic spacing across the slip plane.  If any sets of atoms are closer than this value then the outofplane shift is increased.  Default value is None, which performs no adjustment.

Yields
        
- __a1__ (*float*) The a1 fractional coordinate of a1vect. 

- __a2__ (*float*) The a2 fractional coordinate of a2vect. 

- (*atomman.System*) The fault configuration associated with the a1, a2 shift.

## 4. Examples

### 4.1 fcc {$111$}&lt;$11\bar{2}$&gt; fault path

A 1D path can be generated by only specifying a1vect.

Generate conventional fcc unit cell

In [2]:
a = uc.set_in_units(4.05, 'angstrom')
atoms = am.Atoms(pos=[[0.0, 0.0, 0.0], [0.5, 0.5, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5]])
ucell = am.System(atoms=atoms, box=am.Box.cubic(a), scale=True)

Define parameters for the system

In [3]:
# Specify cutboxvector option
cutboxvector = 'c'

# Specify the slip plane in Miller indices relative to ucell
slipplane = np.array([ 1,  1,  1])

# Specify a small shift to avoid atoms exactly at non-periodic boundary and fault plane
shift = np.array([0.0, 0.0, 0.01])

# Define shift vector in Miller indices relative to ucell
a1vect = 0.5 * np.array([ 1,  1, -2]) 

Use free_surface_basis() to identify rotation uvws

In [4]:
uvws = am.defect.free_surface_basis(slipplane, box=ucell.box, cutboxvector=cutboxvector)
print(uvws)

[[-1.  1.  0.]
 [-1.  0.  1.]
 [ 1.  1.  1.]]


Rotate system, shift and supersize. Note that rotate returns transformation matrix relative to ucell.

In [5]:
system, transform = ucell.rotate(uvws, return_transform=True)
system.atoms.pos += np.dot(shift, system.box.vects)
system.wrap()

system = system.supersize(1, 1, 4)

Initialize the StackingFault object and check parameters

In [6]:
sf = am.defect.StackingFault(system, a1vect=a1vect, ucellbox=ucell.box, transform=transform)

print('Fault system has', sf.system.natoms, 'total atoms.')
print('Fault system has', len(sf.system.atoms.atype[sf.abovefault]), 'atoms above the fault')
print('Fault plane is at z =', sf.faultposcart)
print()
print("Fault system's pbc is", sf.system.pbc)
print()
print('a1vect as given:', sf.a1vect)
print('Cartesian a1vect:', sf.a1vectcart)

Fault system has 48 total atoms.
Fault system has 24 atoms above the fault
Fault plane is at z = 14.029611541307906

Fault system's pbc is [ True  True False]

a1vect as given: [ 0.5  0.5 -1. ]
Cartesian a1vect: [ 2.39320926e-15 -4.96021673e+00  3.91215996e-16]


Generate a single stacking fault configuration with StackingFault.fault()

In [7]:
faultsystem = sf.fault(a1=0.5)
print('atoms in faultsystem are displaced by:')
print(am.displacement(system, faultsystem))

atoms in faultsystem are displaced by:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.66453526e-15  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.000000

Generate configurations along the slip line with StackingFault.iterfaultmap(). Note that the displacements may wrap due to periodic boundaries.

In [8]:
for a1, a2, faultsystem in sf.iterfaultmap(num_a1=10):
    d = am.displacement(system, faultsystem)[-1]
    print('a1 = %.1f, d = [%7.4f %7.4f %7.4f]' % (a1, d[0], d[1], d[2]))

a1 = 0.0, d = [ 0.0000  0.0000  0.0000]
a1 = 0.1, d = [ 0.0000 -0.4960  0.0000]
a1 = 0.2, d = [ 0.0000 -0.9920  0.0000]
a1 = 0.3, d = [ 0.0000 -1.4881  0.0000]
a1 = 0.4, d = [ 0.0000 -1.9841  0.0000]
a1 = 0.5, d = [ 0.0000 -2.4801  0.0000]
a1 = 0.6, d = [ 0.0000 -2.9761  0.0000]
a1 = 0.7, d = [-2.8638  1.4881  0.0000]
a1 = 0.8, d = [-2.8638  0.9920  0.0000]
a1 = 0.9, d = [-2.8638  0.4960  0.0000]


### 4.2 fcc {111} slip plane 2D map

Alternatively, a 2D map for any slip plane can be evaluated by specifying both a1vect and a2vect as non-parallel vectors.

Define parameters for the system. Note that a1vect and a2vect are the two shortest non-parallel Miller vectors in the slip plane.

In [9]:
# Specify cutboxvector option
cutboxvector = 'c'

# Specify the slip plane in Miller indices relative to ucell
slipplane = np.array([ 1,  1,  1])

# Specify a small shift to avoid atoms exactly at non-periodic boundary and fault plane
shift = np.array([0.0, 0.0, 0.01])

# Define shift vectors in Miller indices relative to ucell
a1vect = 0.5 * np.array([-1,  1,  0]) 
a2vect = 0.5 * np.array([-1,  0,  1])

Rotate system, shift and supersize. Note that rotate returns transformation matrix relative to ucell.

In [10]:
uvws = am.defect.free_surface_basis(slipplane, box=ucell.box, cutboxvector=cutboxvector)
system, transform = ucell.rotate(uvws, return_transform=True)
system.atoms.pos += np.dot(shift, system.box.vects)
system.wrap()

system = system.supersize(1, 1, 4)

Initialize the StackingFault object and check parameters

In [11]:
sf = am.defect.StackingFault(system, a1vect=a1vect, a2vect=a2vect, ucellbox=ucell.box, transform=transform)

print('Fault system has', sf.system.natoms, 'total atoms.')
print('Fault system has', len(sf.system.atoms.atype[sf.abovefault]), 'atoms above the fault')
print('Fault plane is at z =', sf.faultposcart)
print()
print("Fault system's pbc is", sf.system.pbc)
print()
print('a1vect as given:', sf.a1vect)
print('Cartesian a1vect:', sf.a1vectcart)
print()
print('a2vect as given:', sf.a2vect)
print('Cartesian a2vect:', sf.a2vectcart)

Fault system has 48 total atoms.
Fault system has 24 atoms above the fault
Fault plane is at z = 14.029611541307906

Fault system's pbc is [ True  True False]

a1vect as given: [-0.5  0.5  0. ]
Cartesian a1vect: [ 2.86378246e+00 -2.75990494e-16 -4.17652603e-16]

a2vect as given: [-0.5  0.   0.5]
Cartesian a2vect: [ 1.43189123e+00  2.48010836e+00 -4.17652603e-16]


Now, StackingFault.iterfaultmap() can generate a 2D map. Note that the &lt;$\bar{1}\bar{1}2$&gt; path corresponds to the points where a1 = a2

In [12]:
for a1, a2, faultsystem in sf.iterfaultmap(num_a1=5, num_a2=5):
    d = am.displacement(system, faultsystem)[-1]
    print('a1 = %.1f, a2 = %.1f, d = [%7.4f %7.4f %7.4f]' % (a1, a2, d[0], d[1], d[2]))

a1 = 0.0, a2 = 0.0, d = [ 0.0000  0.0000  0.0000]
a1 = 0.2, a2 = 0.0, d = [ 0.5728  0.0000  0.0000]
a1 = 0.4, a2 = 0.0, d = [ 1.1455  0.0000  0.0000]
a1 = 0.6, a2 = 0.0, d = [ 1.7183 -0.0000  0.0000]
a1 = 0.8, a2 = 0.0, d = [ 2.2910 -0.0000  0.0000]
a1 = 0.0, a2 = 0.2, d = [ 0.2864  0.4960  0.0000]
a1 = 0.2, a2 = 0.2, d = [ 0.8591  0.4960  0.0000]
a1 = 0.4, a2 = 0.2, d = [ 1.4319  0.4960  0.0000]
a1 = 0.6, a2 = 0.2, d = [ 2.0046  0.4960  0.0000]
a1 = 0.8, a2 = 0.2, d = [ 2.5774  0.4960  0.0000]
a1 = 0.0, a2 = 0.4, d = [ 0.5728  0.9920  0.0000]
a1 = 0.2, a2 = 0.4, d = [ 1.1455  0.9920  0.0000]
a1 = 0.4, a2 = 0.4, d = [ 1.7183  0.9920  0.0000]
a1 = 0.6, a2 = 0.4, d = [ 2.2910  0.9920  0.0000]
a1 = 0.8, a2 = 0.4, d = [ 2.8638  0.9920  0.0000]
a1 = 0.0, a2 = 0.6, d = [ 0.8591  1.4881  0.0000]
a1 = 0.2, a2 = 0.6, d = [ 1.4319  1.4881  0.0000]
a1 = 0.4, a2 = 0.6, d = [ 2.0046  1.4881  0.0000]
a1 = 0.6, a2 = 0.6, d = [ 2.5774  1.4881  0.0000]
a1 = 0.8, a2 = 0.6, d = [-2.5774  1.4881  0.0000]


### 4.3 fcc {111} slip plane using primitive cell

For DFT simulations, it is usually preferred to generate atomic configurations from the primitive unit cells rather than the conventional ones as the primitive cells have fewer atoms.  The challenge with doing so is that the Miller vectors used for rotating and defining the shift vectors need to be relative to the *primitive* cell rather than the conventional one.  This conversion can automatically be handled in free_surface_basis() by specifying the conventional cell setting.

Generate primitive fcc unit cell

In [13]:
a = uc.set_in_units(4.05, 'angstrom')
atoms = am.Atoms(pos=[[0.0, 0.0, 0.0]])
ucell = am.System(atoms=atoms, box=am.Box.trigonal(a/2**0.5, alpha=60), scale=True)

Define parameters for the system relative to the conventional cell.

In [14]:
# Specify cutboxvector option
cutboxvector = 'c'

# Specify the slip plane in Miller indices and the associated conventional cell setting
slipplane = np.array([ 1,  1,  1])
conventional_setting = 'f'

# Specify a small shift to avoid atoms exactly at non-periodic boundary and fault plane
shift = np.array([0.0, 0.0, 0.01])

Use free_surface_basis() to identify rotation uvws

In [15]:
uvws = am.defect.free_surface_basis(slipplane, box=ucell.box, cutboxvector=cutboxvector,
                                    conventional_setting=conventional_setting)
print(uvws)

[[-1.  1.  0.]
 [ 0.  1. -1.]
 [-1. -1. -1.]]


Since free_surface_basis always returns the two shortest in-plane lattice vectors, we can use those for a1vect and a2vect.  

In [16]:
# Note indices are wrong if cutboxvect != 'c'
a1vect = uvws[0]
a2vect = uvws[1]

# Show a1vect and a2vect relative to conventional unit cell
a1vect_conventional = am.tools.miller.vector_primitive_to_conventional(a1vect, setting='f')
a2vect_conventional = am.tools.miller.vector_primitive_to_conventional(a2vect, setting='f')
print('conventional a1vect =', a1vect_conventional)
print('conventional a2vect =', a2vect_conventional)

conventional a1vect = [ 0.  -0.5  0.5]
conventional a2vect = [ 0.5 -0.5  0. ]


Rotate system, shift and supersize. Note that rotate returns transformation matrix relative to ucell.

In [17]:
system, transform = ucell.rotate(uvws, return_transform=True)
system.atoms.pos += np.dot(shift, system.box.vects)
system.wrap()

system = system.supersize(1, 1, 4)

Initialize the StackingFault object and check parameters

In [18]:
sf = am.defect.StackingFault(system, a1vect=a1vect, a2vect=a2vect,
                             ucellbox=ucell.box, transform=transform)

print('Fault system has', sf.system.natoms, 'total atoms.')
print('Fault system has', len(sf.system.atoms.atype[sf.abovefault]), 'atoms above the fault')
print('Fault plane is at z =', sf.faultposcart)
print()
print("Fault system's pbc is", sf.system.pbc)
print()
print('a1vect as given:', sf.a1vect)
print('Cartesian a1vect:', sf.a1vectcart)
print()
print('a2vect as given:', sf.a2vect)
print('Cartesian a2vect:', sf.a2vectcart)

Fault system has 12 total atoms.
Fault system has 6 atoms above the fault
Fault plane is at z = 14.029611541307904

Fault system's pbc is [ True  True False]

a1vect as given: [-1.  1.  0.]
Cartesian a1vect: [ 2.86378246e+00  2.44254078e-16 -1.85055920e-16]

a2vect as given: [ 0.  1. -1.]
Cartesian a2vect: [ 1.43189123e+00  2.48010836e+00 -1.54288963e-16]


This system has the same fault plane position and Cartesian shift vectors as the above example, but 1/4 the atoms!

### 4.4 hcp {0001} slip plane

The system rotation vectors and the shifting vectors can be given in Miller-Bravais indices if the reference unit cell is hexagonal.

Generate hcp unit cell

In [19]:
a = uc.set_in_units(3.18, 'angstrom')
c = uc.set_in_units(5.17, 'angstrom')
atoms = am.Atoms(pos=[[0.0, 0.0, 0.0], [1/3, 2/3, 0.5]])
ucell = am.System(atoms=atoms, box=am.Box.hexagonal(a, c), scale=True)

Define parameters for the system

In [20]:
# Specify cutboxvector option
cutboxvector = 'c'

# Specify the slip plane
slipplane = np.array([ 0,  0,  0,  1])

# Specify a small shift to avoid atoms exactly at non-periodic boundary and fault plane
shift = np.array([0.0, 0.0, 0.01])

# Define shift vectors in Miller indices relative to ucell
a1vect = np.array([ 2, -1, -1,  0]) / 3
a2vect = np.array([ 1,  1, -2,  0]) / 3

Rotate system, shift and supersize. Note that rotate returns transformation matrix relative to ucell.

In [21]:
uvws = am.defect.free_surface_basis(slipplane, box=ucell.box, cutboxvector=cutboxvector)
system, transform = ucell.rotate(uvws, return_transform=True)
system.atoms.pos += np.dot(shift, system.box.vects)
system.wrap()

system = system.supersize(1, 1, 4)

Initialize the StackingFault object and check parameters

In [22]:
sf = am.defect.StackingFault(system, a1vect=a1vect, a2vect=a2vect, ucellbox=ucell.box, transform=transform)

print('Fault system has', sf.system.natoms, 'total atoms.')
print('Fault system has', len(sf.system.atoms.atype[sf.abovefault]), 'atoms above the fault')
print('Fault plane is at z =', sf.faultposcart)
print()
print("Fault system's pbc is", sf.system.pbc)
print()
print('a1vect as given:', sf.a1vect)
print('Cartesian a1vect:', sf.a1vectcart)
print()
print('a2vect as given:', sf.a2vect)
print('Cartesian a2vect:', sf.a2vectcart)

Fault system has 8 total atoms.
Fault system has 4 atoms above the fault
Fault plane is at z = 10.34

Fault system's pbc is [ True  True False]

a1vect as given: [ 0.66666667 -0.33333333 -0.33333333  0.        ]
Cartesian a1vect: [ 3.18000000e+00 -2.73153518e-16  0.00000000e+00]

a2vect as given: [ 0.33333333  0.33333333 -0.66666667  0.        ]
Cartesian a2vect: [1.59       2.75396078 0.        ]


Generate a single stacking fault configuration with StackingFault.fault()

In [23]:
faultsystem = sf.fault(a1=0.1, a2=0.1)
print('atoms in faultsystem are displaced by:')
print(am.displacement(system, faultsystem))

atoms in faultsystem are displaced by:
[[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.88178420e-16]
 [4.77000000e-01 2.75396078e-01 0.00000000e+00]
 [4.77000000e-01 2.75396078e-01 1.77635684e-15]
 [4.77000000e-01 2.75396078e-01 0.00000000e+00]
 [4.77000000e-01 2.75396078e-01 3.55271368e-15]]


### 4.5 DFT support parameters: vacuumwidth and minimum_r

DFT calculations of stacking fault calculations differ from classical atomistics calculatoins in that free surfaces are usually generated by inserting a region of vacuum rather than change boundary conditions, and extensive configuration relaxations are incredibly costly.  The vacuumwidth and minimum_r parameters were designed to support these differences. 

Generate a new StackingFault object using the above parameters plus vacuumwidth

In [24]:
sf = am.defect.StackingFault(system, a1vect=a1vect, a2vect=a2vect, ucellbox=ucell.box, transform=transform, vacuumwidth=10)
print('Original system box:')
print(system.box)
print()
print('Fault system box:')
print(sf.system.box)
print()

Original system box:
avect =  [ 3.180,  0.000,  0.000]
bvect =  [ 1.590,  2.754,  0.000]
cvect =  [ 0.000,  0.000, 20.680]
origin = [ 0.000,  0.000,  0.000]

Fault system box:
avect =  [ 3.180,  0.000,  0.000]
bvect =  [ 1.590,  2.754,  0.000]
cvect =  [ 0.000,  0.000, 30.680]
origin = [ 0.000,  0.000, -5.000]



Generating configurations with minimum_r will adjust the out-of-plane shift component to ensure pairs of atoms across the fault plane are not positioned too close together.

In [25]:
for a1, a2, faultsystem in sf.iterfaultmap(num_a1=5, num_a2=5, minimum_r=a*0.90):
    d = am.displacement(system, faultsystem)[-1]
    print('a1 = %.1f, a2 = %.1f, d = [%7.4f %7.4f %7.4f]' % (a1, a2, d[0], d[1], d[2]))

a1 = 0.0, a2 = 0.0, d = [ 0.0000  0.0000  0.0000]
a1 = 0.2, a2 = 0.0, d = [ 0.6360  0.0000  0.0000]
a1 = 0.4, a2 = 0.0, d = [ 1.2720  0.0000  0.1071]
a1 = 0.6, a2 = 0.0, d = [-1.2720 -0.0000  0.1071]
a1 = 0.8, a2 = 0.0, d = [-0.6360 -0.0000  0.0000]
a1 = 0.0, a2 = 0.2, d = [ 0.3180  0.5508  0.0000]
a1 = 0.2, a2 = 0.2, d = [ 0.9540  0.5508  0.0000]
a1 = 0.4, a2 = 0.2, d = [-1.5900  0.5508  0.0000]
a1 = 0.6, a2 = 0.2, d = [-0.9540  0.5508  0.0000]
a1 = 0.8, a2 = 0.2, d = [-0.3180  0.5508  0.0000]
a1 = 0.0, a2 = 0.4, d = [ 0.6360  1.1016  0.1071]
a1 = 0.2, a2 = 0.4, d = [-0.3180 -1.6524  0.0000]
a1 = 0.4, a2 = 0.4, d = [ 0.3180 -1.6524  0.0000]
a1 = 0.6, a2 = 0.4, d = [-0.6360  1.1016  0.1071]
a1 = 0.8, a2 = 0.4, d = [-0.0000  1.1016  0.1812]
a1 = 0.0, a2 = 0.6, d = [-0.6360 -1.1016  0.1071]
a1 = 0.2, a2 = 0.6, d = [-0.0000 -1.1016  0.0000]
a1 = 0.4, a2 = 0.6, d = [ 0.6360 -1.1016  0.1071]
a1 = 0.6, a2 = 0.6, d = [ 1.2720 -1.1016  0.2533]
a1 = 0.8, a2 = 0.6, d = [ 0.3180  1.6524  0.2533]
